https://photos.app.goo.gl/1J21tCSVy2UHczYE9

multivariate gaussian  
https://en.wikipedia.org/wiki/Multivariate_normal_distribution  
https://www.youtube.com/watch?v=JjB58InuTqM&index=6&list=LLyG2f9f44zZhZr8cLPOa4UQ&t=1s

In [272]:
import os
import pandas as pd
import numpy as np
from glob import glob

# data preprocessing

In [273]:
path_list = glob(os.path.join(os.getcwd(),'dataset','*'))
path_list

['/media/disk3/feynman52/ntu-ml/hw2/dataset/X_test',
 '/media/disk3/feynman52/ntu-ml/hw2/dataset/Y_train',
 '/media/disk3/feynman52/ntu-ml/hw2/dataset/X_train',
 '/media/disk3/feynman52/ntu-ml/hw2/dataset/train.csv',
 '/media/disk3/feynman52/ntu-ml/hw2/dataset/test.csv']

In [274]:
df_Y_train = pd.read_csv(path_list[1])
# df_Y_train

In [275]:
df_X_train = pd.read_csv(path_list[2])
# df_X_train

In [276]:
# list(df_X_train.columns)

In [277]:
df_X_test = pd.read_csv(path_list[0])
# df_X_test

In [278]:
df_train = pd.read_csv(path_list[-2])
# df_train


In [279]:
df_test = pd.read_csv(path_list[-1])
# df_test


In [280]:
def normalize(arr):
    mean = arr.mean(axis=0)
    std = arr.std(axis=0)
    
    # to prevent from divide by inf, std replace 0 with 1
    std[std==0] = 1
    
    arr2 = (arr-mean)/std
    return arr2
    

In [281]:
y_train = df_Y_train.values
x_train = df_X_train.values
x_test = df_X_test.values
y_train.shape,x_train.shape,x_test.shape

((32561, 1), (32561, 106), (16281, 106))

In [282]:
y_train = y_train.astype('float')
x_train = x_train.astype('float')
x_test = x_test.astype('float')

# y_train = normalize(y_train)
y_train = y_train.reshape(-1)

x_train = normalize(x_train)
x_test = normalize(x_test)

print(y_train.shape,x_train.shape,x_test.shape,)
print(y_train[0] ,x_train[0,0] ,x_test[0,0] )

(32561,) (32561, 106) (16281, 106)
0.0 0.030670557354391753 -0.9941292592084697


In [283]:
def shuffle(arr1,arr2):
    id_list = np.arange(len(arr1))
    np.random.shuffle(id_list)
    
    arr1_random = arr1[id_list]
    arr2_random = arr2[id_list]
    return arr1_random,arr2_random
    
x_train,y_train = shuffle(x_train,y_train)
x_train.shape,y_train.shape

((32561, 106), (32561,))

# train split to (train,val)

In [284]:
# split (train,val)
split_percent = 0.1
num_sample = len(y_train)
num_sample_train = int(num_sample*(1-split_percent))
x_train_split = x_train[:num_sample_train]
y_train_split = y_train[:num_sample_train]
x_val_split = x_train[num_sample_train:]
y_val_split = y_train[num_sample_train:]
x_train_split.shape,y_train_split.shape,x_val_split.shape,y_val_split.shape

((29304, 106), (29304,), (3257, 106), (3257,))

# training data split to two groups
y = 1, group 1 !!!  
y = 0, group 2 !!!

In [285]:
x_group_1 = []
y_group_1 = []
x_group_2 = []
y_group_2 = []

for i in range(len(y_train_split)):
    if y_train_split[i] == 1:
        x_group_1.append(x_train_split[i])
        y_group_1.append(x_train_split[i])
    else:
        x_group_2.append(x_train_split[i])
        y_group_2.append(x_train_split[i])
        
x_group_1 = np.array(x_group_1)
y_group_1 = np.array(y_group_1)
x_group_2 = np.array(x_group_2)
y_group_2 = np.array(y_group_2)

x_group_1.shape,y_group_1.shape,x_group_2.shape,y_group_2.shape,

((7038, 106), (7038, 106), (22266, 106), (22266, 106))

# calculate mu and sigma

In [286]:
mu1 = np.mean(x_group_1,axis=0)
mu2 = np.mean(x_group_2,axis=0)

print(mu1[0],mu2[0])
print(mu1.shape,mu2.shape)


0.4166293324350826 -0.13469597521881027
(106,) (106,)


In [287]:

x_group_1_norm = x_group_1 - mu1
x_group_2_norm = x_group_2 - mu2

print(x_group_1_norm.shape,x_group_2_norm.shape)
print(x_group_1[0,0],x_group_2[0,0])
print(x_group_1_norm[0,0],x_group_2_norm[0,0])

(7038, 106) (22266, 106)
1.0570467320885653 2.230048074641906
0.6404173996534827 2.3647440498607164


In [288]:
n1 = len(x_group_1_norm)
n2 = len(x_group_2_norm)
n1,n2

(7038, 22266)

In [289]:
dim = x_train.shape[1]
dim

106

In [290]:
# i,j=1,2
# x_group_1_norm[:,i] @ x_group_1_norm[:,j],np.dot(x_group_1_norm[:,i],x_group_1_norm[:,j])

In [291]:
# for i in range(3):
#     print('-----')
#     for j in range(i,3):
#         print(i,j)
        

In [292]:
sigma1 = np.zeros(shape=[dim,dim])
sigma2 = np.zeros(shape=[dim,dim])

for i in range(dim):
    for j in range(i,dim):        
        # group1
        xi = x_group_1_norm[:,i]
        xj = x_group_1_norm[:,j]
        sigma1[i,j] = (xi @ xj) / n1
        sigma1[j,i] = sigma1[i,j]
        
        # group2
        xi = x_group_2_norm[:,i]
        xj = x_group_2_norm[:,j]
        sigma2[i,j] = (xi @ xj) / n2
        sigma2[j,i] = sigma2[i,j]

print(sigma1.shape,sigma2.shape)


(106, 106) (106, 106)


In [293]:
# sigma1.shape,sigma2.shape
# sigma1[0],sigma2[0]
# sigma1[:,0],sigma2[:,0]

In [294]:
sigma = (n1/(n1+n2))*sigma1 + (n2/(n1+n2))*sigma2
# sigma.shape,sigma[0]

In [295]:
sigma_inverse = np.linalg.inv(sigma)
sigma_inverse.shape

(106, 106)

# calculate w and b

In [296]:
w = (mu1 - mu2) @ sigma_inverse
w.shape

(106,)

In [297]:
b = (- 0.5 * mu1.reshape(1,-1) @ sigma_inverse @ mu1 
     - 0.5 * mu2.reshape(1,-1) @ sigma_inverse @ mu2 
     + np.log(n1/n2) )
b.shape

(1,)

# validate

In [298]:
z_val_predict = x_val_split @ w + b
# (3257,) = (3257,106) @ (106,) + (1,)

z_val_predict.shape,z_val_predict[:2],(x_val_split @ w)[:2],b[0]

((3257,),
 array([-3.87378248,  1.27105258]),
 array([-1.71044836,  3.4343867 ]),
 -2.163334123869173)

In [299]:
def sigmoid(z):
    res = 1 / (1.0 + np.exp(-z))
    return np.clip(res, 1e-8, 1-(1e-8))

y_val_predict = sigmoid(z_val_predict)
y_val_predict.shape,y_val_predict[:5]

((3257,), array([0.02035662, 0.78092288, 0.06336145, 0.41238001, 0.17208333]))

# calculate acc


In [300]:
y_val_predict_category = np.round(y_val_predict)
compare = (y_val_predict_category == y_val_split)
accuracy = np.sum(compare) / len(x_val_split)
accuracy    

0.842493091802272

# calculate cross_entropy

In [301]:
y_real_0 = y_val_split
y_real_1 = 1-y_val_split
y_predict_0 = y_val_predict
y_predict_1 = 1-y_val_predict
cross_entropy = - y_real_0 * np.log(y_predict_0) - y_real_1 * np.log(y_predict_1) 
mean_cross_entropy = np.mean(cross_entropy)
mean_cross_entropy

0.34310682695446265